[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df_IMDB_rating = pd.read_csv('labeledTrainData.csv', encoding='latin-1', sep='\t')
df_IMDB_rating.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
len(df_IMDB_rating)

25000

In [4]:
# Check duplicate values
df_IMDB_rating.duplicated().values.any()

False

In [5]:
#check missing values
df_IMDB_rating.isnull().values.any()

False

## 2. Preprocessing

In [6]:
# stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words[:5])

['i', 'me', 'my', 'myself', 'we']


In [7]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    # remove html like "<abc>" -> ""
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # when lower text, all non-word chars will remain in uppercase.
    text = re.sub('[\W]+', ' ', text.lower())
    
    # Removing unnecessary spaces
    while text.find('  ') != -1:
        text = text.replace('  ', ' ')
    text = text.strip()
    
    return text

# Testcase
print(preprocessor('   Make it work :)), ||| <b>with love</b> ||  \American Psycho,\"'))

make it work with love american psycho


In [8]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] #stem for each word splitted by a space

# Testcase
print(tokenizer_porter('Hi there, I am doing this, do it, does it...'))

['Hi', 'there,', 'I', 'am', 'do', 'this,', 'do', 'it,', 'doe', 'it...']


In [49]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = df_IMDB_rating['review']
y = df_IMDB_rating['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

# The dimension of X_train and X_test will change to 2-d after applying tfidf for vectorization.
print("X_train_shape: {0}, Y_train_shape: {1}".format(X_train.shape, y_train.shape))
print("X_test_shape: {0}, Y_test_shape: {1}".format(X_test.shape, y_test.shape))
print("X_train_samples: {0},\nY_train_samples: {1}".format(X_train[0], y_train[0]))
print("X_test_samples: {0},\nY_test_samples: {1}".format(list(X_test)[3], list(y_test)[3]))

X_train_shape: (20000,), Y_train_shape: (20000,)
X_test_shape: (5000,), Y_test_shape: (5000,)
X_train_samples: With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made i

## 3.1 Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Use tfidf to vectorize reviews.
tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor,
                        ngram_range=(1,1),
                        min_df=5)

# Make a pipeline including using tfidf first, then logistic regression to make a classifier.
# This is used for both training and testing with the same input format
# X is list of text
# y is list of sentiment (0,1) 
clf = Pipeline([('vec', tfidf),
                ('clf', LogisticRegression(random_state=0))])

## 3.2. Tuning hyper-params

### 3.3.1. Cross-validation, it is convenient but slow for large training data with 20000 samples
Currently, using this way

In [13]:
# from sklearn.model_selection import cross_val_score

# #0.88445: ngram_range=(1,1)
# #0.88469: ngram_range=(1,1), min_df=5

# # Currently, tuning hyper-params by hand
# scores = cross_val_score(clf, X_train, y_train, cv=5)
# print(np.mean(scores))

0.8846999999999999


### 3.3.2. The better way for time-cost: Split Testset into two parts Testset and Validation dataset (ratio 1:1)
At the moment, not using this method

In [41]:
print("y_train_neg/y_train_pos = {0}/{1}".format(len(np.where(y_train == 0)[0]), len(np.where(y_train == 1)[0])))
X_dev, X_test_new, y_dev, y_test_new = train_test_split(X_test, y_test, test_size=0.5, random_state=102)
print("X_dev_shape: {0}, Y_dev_shape: {1}".format(X_dev.shape, y_dev.shape))
print("X_test_shape: {0}, Y_test_shape: {1}".format(X_test_new.shape, y_test_new.shape))
print("y_dev_neg/y_dev_pos = {0}/{1}".format(len(np.where(y_dev == 0)[0]), len(np.where(y_dev == 1)[0])))

y_train_neg/y_train_pos = 9997/10003
X_dev_shape: (2500,), Y_dev_shape: (2500,)
X_test_shape: (2500,), Y_test_shape: (2500,)
y_dev_neg/y_dev_pos = 1265/1235


## 3.3. Retrain the model with the best pick-up hyper-params

In [42]:
# Use tfidf to vectorize reviews.
tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor,
                        ngram_range=(1,1),
                        min_df=5)

# Make a pipeline including using tfidf first, then logistic regression to make a classifier.
clf = Pipeline([('vec', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=5, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x000001AE934FF558>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x000001AE8F571AF8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
      

## 4. Evaluate Model

In [43]:
# Using Test dataset to evaluate model.
# classification_report.
# confusion matrix.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.885
confusion matrix:
 [[2160  343]
 [ 232 2265]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      2503
           1       0.87      0.91      0.89      2497

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.88      5000
weighted avg       0.89      0.89      0.88      5000



## 4.1. Some test cases

In [47]:
IMDB_rating = list(X_test[:5])

preds = clf.predict_proba(IMDB_rating)

for i in range(len(IMDB_rating)):
    print(f'{IMDB_rating[i]} --> Negative, Positive = {preds[i]} --> True label = {list(y_test)[i]}')

Great acting on the part of Gretchen Mol. This film is one of the best biopics to hit the screen in some time. While it does cover the majority of Bettie's young life, it also manages to stay on a mostly focused path which is something most biographical films seem to lack. There is some lovely and alarmingly funny subtext in the dialogue and acting. This film is an excellent break from the Dir. of \American Psycho,\" and I think this will show through as her best work to date. Oh, and as a cinematography buff, I give this film 100% in the cine dept. It was amazing how well they pulled off a 50s look with modern film stocks. Accolades to the D.O.P. All around very enjoyable. I recommend any interested to see it: 8/10." --> Negative, Positive = [0.04317376 0.95682624] --> True label = 1
While this movie isn't a classic by any stretch, it is very entertaining as I remember it. I saw it about 15 years ago on HBO and loved the movie. It was written by the same guy that wrote and directed \A

## 5.1 Export Model 

In [45]:
# Using pickle to export our trained model.
import pickle
import os

pickle.dump(clf, open('LR_IMDB_rating.pkl', 'wb'))

## 5.2 Load Model

In [46]:
# Loading the model.
with open('LR_IMDB_rating.pkl', 'rb') as model:
    LR_IMDB_rating = pickle.load(model)
    
preds = LR_IMDB_rating.predict_proba(IMDB_rating)

for i in range(len(IMDB_rating)):
    print(f'{IMDB_rating[i]} --> Negative, Positive = {preds[i]} --> True label = {list(y_test)[i]}')

Great acting on the part of Gretchen Mol. This film is one of the best biopics to hit the screen in some time. While it does cover the majority of Bettie's young life, it also manages to stay on a mostly focused path which is something most biographical films seem to lack. There is some lovely and alarmingly funny subtext in the dialogue and acting. This film is an excellent break from the Dir. of \American Psycho,\" and I think this will show through as her best work to date. Oh, and as a cinematography buff, I give this film 100% in the cine dept. It was amazing how well they pulled off a 50s look with modern film stocks. Accolades to the D.O.P. All around very enjoyable. I recommend any interested to see it: 8/10." --> Negative, Positive = [0.04317376 0.95682624] --> True label = 1
While this movie isn't a classic by any stretch, it is very entertaining as I remember it. I saw it about 15 years ago on HBO and loved the movie. It was written by the same guy that wrote and directed \A

In [ ]:
Thank you